In [1]:
from rosbags.rosbag2 import Reader
from rclpy.serialization import deserialize_message
from beep_interfaces.msg import MotorData, TuningValues, Setpoints
from std_msgs.msg import Float32

from pathlib import Path
from glob import glob
from pid_helper import IMU_ANGLE_ERROR

import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:

class TopicMeta:
    def __init__ (self, name, type):
        self.name = name
        self.type = type

topic_metas = {
    "/dev0/motor_data": TopicMeta("motor0", MotorData),
    "/dev1/motor_data": TopicMeta("motor1", MotorData),
    "/output/lean_tuning_values": TopicMeta("lean_tuning", TuningValues),
    "/output/lean_angle": TopicMeta("lean_angle", Float32),
    "/output/turn_tuning_values": TopicMeta("turn_tuning", TuningValues),
    "/output/turn_angle": TopicMeta("turn_angle", Float32),
    "/setpoints": TopicMeta("setpoints", Setpoints),
    "/dev0/duty": TopicMeta("duty", Float32)
}

def get_all_data(path):
    data = {}
    timestamps = {}
    
    #read from rosbags
    with Reader(path) as reader:
        for connection in reader.connections:
            if connection.topic in topic_metas.keys():
                print(connection.topic, connection.msgtype)
                name = topic_metas[connection.topic].name
                data[name] = []
                timestamps[name] = []

        for connection, timestamp, rawdata in reader.messages():
            if connection.topic in topic_metas:
                meta = topic_metas[connection.topic]
                msg = deserialize_message(rawdata, meta.type)
                data[meta.name].append(msg)
                timestamps[meta.name].append(timestamp)

    #turn timestamps to time
    time = timestamps
    for key, val in timestamps.items():
        time[key] = np.array(val)
        
    return data, time

def get_graphable_data(data, time, source, attribute="data", getTimestamps = False):
    r_time = time[source]
    if (getTimestamps==False):
        r_time = (time[source] - time[source][0]) / 1e9
        
    return np.array(list(map(lambda x: getattr(x, attribute), data[source]))), r_time

def synch_timestamps (timestamps):
    earliest = min(list(map(lambda x: x[0], timestamps)))
    times = list(map(lambda x: (x - earliest) / 1e9, timestamps))
    return times

In [4]:
test1_path = Path("../../important_data/10-14/center_kick/src/important_data/10-14/center_kick/rosbag2_2024_10_12-14_42_35_0.db3")
data, time = get_all_data(test1_path)


NameError: name 'path' is not defined